In [2]:
import sys
sys.path.append("..")

import uuid
import logging
import argparse
import re
import json
import jieba
import math
import os
import datetime
import time
import collections
from agent import Agent
from typing import List, Dict, Any, Tuple
from LLM.deli_client import search_law
from LLM.apillm import APILLM
from rouge_chinese import Rouge
from LLM.api_pool.api_pool import api_pool

/home/bjtuzky/.conda/envs/AIGC/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
true_path='/home/swh/legal/crime_data'
cases=[]
id=0
with open(os.path.join(true_path,'crime_data_1.json')) as f:
    for line in f:
        case = json.loads(line)
        if id>=1 and id<=8:
            cases.append(case)
            print(case)
        if id==99:
            cases.append(case)
            print(case)
        if len(cases)==9:
            break
        id+=1
# print(len(cases))

tmp=cases[8]
lst=[{"content":"拘役4个月"},{"content":"有期徒刑1年6个月"},tmp]
for dict in cases:
    lst.append(dict)
cases=lst[:-2]
print(len(cases))

{'id': '（2018）京0117刑初202号', 'type': '判决书', 'program': '刑事一审', 'claim': '张亮交通肇事一审刑事判决书北京市平谷区人民法院刑事判决书（2018）京0117刑初202号：北京市平谷区人民检察院以京平检公诉刑诉［2018］207号起诉书指控被告人张亮犯交通肇事罪，于2018年7月27日向本院提起公诉。', 'fact': '经审理查明，2018年6月25日22时许，被告人张亮持与准驾车型不符的驾驶证，酒后驾驶红色嘉陵牌普通两轮摩托车（车牌号：×××），载乘被害人魏某、王某由南向北行驶至北京市平谷区某处时，两轮摩托车倒地，致张亮、魏某、王某受伤，被害人魏某于2018年6月30日经医院抢救无效死亡。经北京市公安交通司法鉴定中心检验，张亮血液中的酒精含量为96.5ｍｇ／100ｍｌ。经北京市公安局平谷分局交通支队认定，张亮负此事故的全部责任。2018年6月28日，被告人张亮经民警电话通知后主动到公安机关接受讯问，并如实供述了上述事实。本案在审理过程中，被害人魏某的家属表示另行起诉解决民事赔偿问题，并要求对被告人张亮依法处理；被害人王某表示不要求追究被告人张亮的刑事责任。', 'reason': '本院认为，被告人张亮违反交通运输管理法规，因而发生重大事故，致1人死亡，其行为已构成交通肇事罪，应予惩处。公诉机关指控的罪名成立。鉴于被告人张亮具有自首情节，故可对其依法从轻处罚。', 'result': '被告人张亮犯交通肇事罪，判处有期徒刑一年六个月（刑期从判决执行之日起计算，判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2018年6月28日起至2019年12月27日止）。', 'content': '张亮交通肇事一审刑事判决书北京市平谷区人民法院刑事判决书（2018）京0117刑初202号：北京市平谷区人民检察院以京平检公诉刑诉［2018］207号起诉书指控被告人张亮犯交通肇事罪，于2018年7月27日向本院提起公诉。本院依法适用简易程序，实行独任审判，公开开庭进行了审理。北京市平谷区人民检察院指派检察员廖璐出庭支持公诉，被告人张亮到庭参加诉讼。现已审理终结经审理查明，2018年6月25日22时许，被告人张亮持与准驾车型不符的驾驶证，酒后驾驶红色嘉陵牌普通两轮摩托车（车牌号：×××），载乘被害人魏某、王某由南

In [4]:
for case in cases:    
    print(case)

{'content': '拘役4个月'}
{'content': '有期徒刑1年6个月'}
{'id': '（2018）辽0103刑初660号', 'type': '判决书', 'program': '刑事一审', 'claim': '徐东东危险驾驶罪一审刑事判决书沈阳市沈河区人民法院刑事判决书（2018）辽0103刑初660号：沈阳市沈河区人民检察院以沈河检公诉刑诉〔2018〕601号起诉书指控被告人徐东东犯危险驾驶罪一案，于2018年7月30日向本院提起公诉。', 'fact': '沈阳市沈河区人民检察院指控，2017年11月5日23时02分，饮酒后驾驶号牌为辽Ｆ×××××号奥迪牌小型轿车在沈阳市沈河区处被交警查获。经鉴定，在徐东东静脉血中检出酒精（乙醇），其含量为105.2ｍｇ／100ｍｌ。', 'reason': '本院认为，公诉机关601号起诉书指控被告人徐东东危险驾驶犯罪的事实清楚，证据确实充分，指控罪名成立。被告人徐东东犯罪后能够如实供述自己的罪行，依法予以从轻处罚。被告人徐东东还具有以下从轻处罚情节，被告人徐东东积极交纳罚金，具有悔罪表现，可酌情从轻处罚。', 'result': '被告人徐东东犯危险驾驶罪，判处拘役一个月，并处罚金人民币一万元，依法上缴国库。（刑期从判决执行之日起计算，判决执行以前先行羁押的，羁押一日折抵刑期一日，即自2018年8月3日起至2018年8月28日止。', 'content': '徐东东危险驾驶罪一审刑事判决书沈阳市沈河区人民法院刑事判决书（2018）辽0103刑初660号：沈阳市沈河区人民检察院以沈河检公诉刑诉〔2018〕601号起诉书指控被告人徐东东犯危险驾驶罪一案，于2018年7月30日向本院提起公诉。本院依法适用简易程序，实行独任审判，公开开庭审理了本案。沈阳市沈河区人民检察院指派检察员张某出庭支持公诉。被告人徐东东到庭参加诉讼，现已审理终结沈阳市沈河区人民检察院指控，2017年11月5日23时02分，饮酒后驾驶号牌为辽Ｆ×××××号奥迪牌小型轿车在沈阳市沈河区处被交警查获。经鉴定，在徐东东静脉血中检出酒精（乙醇），其含量为105.2ｍｇ／100ｍｌ。被告人徐东东对上述事实，随案移送证据均无异议本院认为，公诉机关601号起诉书指控被告人徐东东危险驾驶犯罪的事实清楚，证据确实充分，指控罪名成立。被告人徐东东犯罪后能

In [ ]:
import json
pure_predict_path='../result/pure_predict'
court_predict_path='../result/test_result'

llms=os.listdir(pure_predict_path)

for llm in llms:
    lst=os.listdir(os.path.join(court_predict_path,llm))
    dict={}
    for name in lst:
        num=name[5]
        dict[num]=name
    print(llm,dict)
    for id in range(10):
        # print(os.path.join(pure_predict_path,llm,f'case_{id}.json'))
        with open(os.path.join(pure_predict_path,llm,f'case_{id}.json'), "r", encoding="utf-8") as f:
            ppred=json.load(f)['judge']
        # print(dict[str(id)])
        # print(os.path.join(court_predict_path,llm,dict[str(id)]))
        with open(os.path.join(court_predict_path,llm,dict[str(id)]),"r", encoding="utf-8") as f:
            cpred = json.load(f)
        cpred=cpred[-5]['content']
        # print(ppred,cpred)
        
        

claude-3-sonnet {'4': 'case_4_1218_2259.json', '3': 'case_3_1218_2316.json', '1': 'case_1_1218_2326.json', '9': 'case_9_1218_2227.json', '0': 'case_0_1212_1659.json', '8': 'case_8_1218_2231.json', '7': 'case_7_1218_2235.json', '2': 'case_2_1212_1654.json', '5': 'case_5_1218_2252.json', '6': 'case_6_1218_2248.json'}
根据案情，判决书如下：

××××人民法院
刑事判决书
（××××）×刑初字第××号

公诉机关××××人民检察院。
被告人张某某，男，1985年3月13日出生于山东省××××，汉族，初中文化，公民身份证号码37032419850313XXXX，户籍地山东省××××。因涉嫌故意伤害罪，于2023年6月16日被刑事拘留，同年6月19日取保候审。

××××人民检察院于2023年6月20日以被告人张某某犯故意伤害罪，向本院提起公诉。本院受理后，依法组成合议庭，公开开庭审理了本案。××××人民检察院检察员×××出庭支持公诉，被告人张某某及其辩护人×××到庭参加诉讼。本案现已审理终结。

经审理查明，2022年11月28日，被告人张某某开车至山东省××××南头河东侧时，与该位置正在装卸货的人员田某某等人因停车问题发生争执。后被告人驾车离开，并在驾驶室位置向后扔出一个玻璃瓶，将被害人田某某嘴部砸伤，造成轻伤二级。事后，被告人张某某赔偿了被害人田某某2万元，田某某出具了谅解书。上述事实，有被告人张某某的供述，证人证言，勘验笔录，医疗诊断等证据证实。

本院认为，被告人张某某因与他人发生争执，故意投掷玻璃瓶伤害他人身体，其行为已构成故意伤害罪。虽然被告人张某某赔偿了被害人经济损失，被害人也出具了谅解书，但犯罪行为已造成事实上的危害结果，需要承担相应的刑事责任。鉴于被告人张某某赔偿了被害人经济损失，被害人予以谅解，且系初犯，依法可从轻处罚。依照《中华人民共和国刑法》第二百三十四条第一款之规定，判决如下：

被告人张某某犯故意伤害罪，判处有期徒刑四个月，缓刑六个

In [18]:
import json
with open('./dd.json',"r",encoding="utf-8") as f:
    lst=json.load(f)
print(lst)

[{'d': 344}]
